In [1]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [2]:
SEASONS = list(range(2016, 2023))

In [3]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [4]:
async def get_html(url, selector, sleep = 5, retries = 3):
    html = None
    for i in range(1, retries + 1):
        time.sleep(sleep * i)

        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"TimeoutError on {url}")
            continue
        else:
            break
    return html

In [5]:
async def scrape_season(season):
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    html = await get_html(url, "#content .filter")
    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    standings_pages = [f"https://basketball-reference.com{l}" for l in href]

    for url in standings_pages:
        save_path = os.path.join(STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#all_schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [6]:
for season in SEASONS:
    await scrape_season(season)

2015-16 NBA Schedule | Basketball-Reference.com
2016-17 NBA Schedule | Basketball-Reference.com
2017-18 NBA Schedule | Basketball-Reference.com
2018-19 NBA Schedule | Basketball-Reference.com
2019-20 NBA Schedule | Basketball-Reference.com
2020-21 NBA Schedule | Basketball-Reference.com
2021-22 NBA Schedule | Basketball-Reference.com


In [7]:
standings_files = os.listdir(STANDINGS_DIR)

In [8]:
async def scrape_game(standings_files):
    with open(standings_files, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and "boxscore" in l and ".html" in l]
    box_scores = [f"https://www.basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        if not html:
            continue
        with open(save_path, "w+") as f:
            f.write(html)

In [9]:
for f in standings_files:
    filepath = os.path.join(STANDINGS_DIR, f)

    await scrape_game(filepath)

Spurs vs Pelicans, January 22, 2020 | Basketball-Reference.com
Suns vs Mavericks, January 28, 2020 | Basketball-Reference.com
Jazz vs Spurs, January 29, 2020 | Basketball-Reference.com
Rockets vs Trail Blazers, January 29, 2020 | Basketball-Reference.com
Raptors vs Cavaliers, January 30, 2020 | Basketball-Reference.com
Raptors vs Pistons, January 31, 2020 | Basketball-Reference.com
Nuggets vs Spurs, August 5, 2020 | Basketball-Reference.com
76ers vs Wizards, August 5, 2020 | Basketball-Reference.com
Magic vs Celtics, August 9, 2020 | Basketball-Reference.com
76ers vs Trail Blazers, August 9, 2020 | Basketball-Reference.com
Rockets vs Kings, August 9, 2020 | Basketball-Reference.com
Nuggets vs Lakers, August 10, 2020 | Basketball-Reference.com
Nets vs Magic, August 11, 2020 | Basketball-Reference.com
Raptors vs 76ers, August 12, 2020 | Basketball-Reference.com
2020 NBA Western Conference First Round Game 5: Jazz vs Nuggets, August 25, 2020 | Basketball-Reference.com
2020 NBA Western Con